In [3]:
!git clone https://github.com/NeyoxDrago/JantaHack-Mobility-Analytics.git

fatal: destination path 'JantaHack-Mobility-Analytics' already exists and is not an empty directory.


In [157]:
import pandas as pd
train = pd.read_csv("JantaHack-Mobility-Analytics/train.csv")
test = pd.read_csv("JantaHack-Mobility-Analytics/test.csv")
TARGET = "Surge_Pricing_Type"
df = pd.concat([train,test]).reset_index(drop=True)
df

,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender,Surge_Pricing_Type
0,T0005689460,6.77,B,1.0,2.42769,A,A,3.90500,0,40.0,46,60,Female,2.0
1,T0005689461,29.47,B,10.0,2.78245,B,A,3.45000,0,38.0,56,78,Male,2.0
2,T0005689464,41.58,NaN,10.0,NaN,NaN,E,3.50125,2,NaN,56,77,Male,2.0
3,T0005689465,61.56,C,10.0,NaN,NaN,A,3.45375,0,NaN,52,74,Male,3.0
4,T0005689467,54.95,C,10.0,3.03453,B,A,3.40250,4,51.0,49,102,Male,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219052,T0005908503,29.43,C,3.0,2.40720,C,A,0.99750,0,74.0,48,72,Female,NaN
219053,T0005908504,14.83,C,10.0,2.57009,A,D,4.74375,0,109.0,42,60,Female,NaN
219054,T0005908505,37.86,A,6.0,3.08227,C,A,1.63500,0,NaN,57,77,Male,NaN
219055,T0005908511,10.63,C,1.0,NaN,NaN,J,2.59875,1,107.0,51,77,Male,NaN


In [158]:
df.isnull().sum()

Trip_ID                             0
Trip_Distance                       0
Type_of_Cab                     33368
Customer_Since_Months            9886
Life_Style_Index                33520
Confidence_Life_Style_Index     33520
Destination_Type                    0
Customer_Rating                     0
Cancellation_Last_1Month            0
Var1                           117819
Var2                                0
Var3                                0
Gender                              0
Surge_Pricing_Type              87395
dtype: int64

In [0]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=6)
df['Life_Style_Index'] = imputer.fit_transform(df[['Life_Style_Index']])
df['Var1'] = imputer.fit_transform(df[['Var1']])
df['Customer_Since_Months'] = imputer.fit_transform(df[['Customer_Since_Months']])

In [166]:
df.isnull().sum()

Trip_ID                            0
Trip_Distance                      0
Type_of_Cab                    33368
Customer_Since_Months              0
Life_Style_Index                   0
Confidence_Life_Style_Index    33520
Destination_Type                   0
Customer_Rating                    0
Cancellation_Last_1Month           0
Var1                               0
Var2                               0
Var3                               0
Gender                             0
Surge_Pricing_Type             87395
dtype: int64

In [0]:
import numpy as np 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

def impute(df,column):
  m=dict((w,i) for i,w in enumerate(set(df[column].dropna())))
  knn_imputer = KNNImputer()
  imputed_values = knn_imputer.fit_transform([df[column].map(d).values])[0]
  return imputed_values

df['Type_of_Cab_imputed'] = impute(df,"Type_of_Cab")
df['Confidence_Life_Style_Index_imputed'] = impute(df,"Confidence_Life_Style_Index")

In [0]:
df.to_csv("all_data.csv",index=None)

In [3]:
!unrar e all_data.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from all_data.rar


Would you like to replace the existing file all_data.csv
3145728 bytes, modified on 2020-05-17 16:38
with a new one
18261926 bytes, modified on 2020-05-17 22:05

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit Y

Extracting  all_data.csv                                                   0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76%

In [0]:
import pandas as pd
merged_data =pd.read_csv("all_data.csv")

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
merged_data['Gender'] = encoder.fit_transform(merged_data['Gender'])

merged_data['Type_of_Cab_imputed'] = merged_data['Type_of_Cab_imputed'].apply(lambda x:int(x))
merged_data['Confidence_Life_Style_Index_imputed'] = merged_data['Confidence_Life_Style_Index_imputed'].apply(lambda x:int(x))


train = merged_data[~merged_data.Surge_Pricing_Type.isna()] 
test = merged_data[merged_data.Surge_Pricing_Type.isna()] 

In [5]:
!pip install catboost

     |████████████████████████████████| 64.7MB 61kB/s 


In [0]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.metrics import accuracy_score

In [0]:
drop_cols = ["Type_of_Cab","Confidence_Life_Style_Index"]
train_data,test_data = train_test_split(train.drop(drop_cols,axis=1),test_size=0.2,shuffle=True,random_state=11)

TARGET = "Surge_Pricing_Type"

In [123]:
X = train_data.drop([TARGET,"Trip_ID"],axis=1)
Y = train_data[TARGET]

X_test = test_data.drop([TARGET,"Trip_ID"],axis=1)
Y_test = test_data[TARGET]

cat_params ={
    "iterations":2000,
    # 'random_seed': 7,
    "od_type":"Iter",
    "eval_metric":"Accuracy",
    "task_type":"GPU",
    "verbose":100,
    "early_stopping_rounds":400,
    # "boosting_type":'Plain'
}

model = CatBoostClassifier(cat_features=[3,7,8,9,10,11],**cat_params)
model.fit(X,Y,eval_set=(X_test,Y_test))
score = accuracy_score(Y_test,model.predict(X_test))
print("Score : ",score)

Learning rate set to 0.124325
0:	learn: 0.6798792	test: 0.6792997	best: 0.6792997 (0)	total: 30ms	remaining: 1m
100:	learn: 0.7059499	test: 0.6997304	best: 0.7000722 (98)	total: 2.18s	remaining: 40.9s
200:	learn: 0.7105735	test: 0.7018570	best: 0.7018570 (200)	total: 4.23s	remaining: 37.8s
300:	learn: 0.7139439	test: 0.7025785	best: 0.7026545 (297)	total: 6.33s	remaining: 35.8s
400:	learn: 0.7167921	test: 0.7033000	best: 0.7033000 (400)	total: 8.46s	remaining: 33.7s
500:	learn: 0.7200581	test: 0.7028823	best: 0.7035279 (434)	total: 10.7s	remaining: 31.9s
600:	learn: 0.7228778	test: 0.7019709	best: 0.7035279 (434)	total: 13s	remaining: 30.3s
700:	learn: 0.7261913	test: 0.7024266	best: 0.7035279 (434)	total: 15.4s	remaining: 28.6s
800:	learn: 0.7286882	test: 0.7027304	best: 0.7035279 (434)	total: 18s	remaining: 26.9s
bestTest = 0.7035278928
bestIteration = 434
Shrink model to first 435 iterations.
Score :  0.7035278927581362


# Testing Time

In [13]:
test.dropna(axis=1,inplace=True)
test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Trip_ID,Trip_Distance,Customer_Since_Months,Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender,Type_of_Cab_imputed,Confidence_Life_Style_Index_imputed
131662,T0005689459,9.44,10.0,2.57438,B,3.68000,2,64.095972,46,63,1,2,2
131663,T0005689462,32.15,10.0,2.85143,A,1.59375,0,65.000000,49,80,0,1,2
131664,T0005689463,10.38,4.0,2.70530,D,4.50500,0,64.095972,47,74,1,5,2
131665,T0005689466,14.94,6.0,2.48159,E,4.53000,0,63.000000,43,54,1,0,5
131666,T0005689468,32.03,7.0,2.81598,K,4.60125,3,96.000000,44,56,1,1,2


In [124]:
predictions = model.predict(test.drop("Trip_ID",axis=1))
submission = pd.DataFrame(predictions,columns=['Surge_Pricing_Type'])
submission['Trip_ID']=test["Trip_ID"].values
submission.to_csv("0.70352_iter_submission.csv",index=None)
submission

,Surge_Pricing_Type,Trip_ID
0,1.0,T0005689459
1,2.0,T0005689462
2,2.0,T0005689463
3,2.0,T0005689466
4,2.0,T0005689468
...,...,...
87390,2.0,T0005908503
87391,2.0,T0005908504
87392,1.0,T0005908505
87393,2.0,T0005908511
